In [2]:
from ultralytics import YOLO
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import ultralytics
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
import glob
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image, ImageEnhance, ImageOps
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [3]:
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"Ultralytics 버전: {ultralytics.__version__}")
print(f"CUDA 버전: {torch.version.cuda}")

model = YOLO('yolov8n.pt')
print(f"모델 작업 유형: {model.task}")

PyTorch 버전: 2.0.1+cu117
CUDA 사용 가능: True
Ultralytics 버전: 8.3.0
CUDA 버전: 11.7
모델 작업 유형: detect


In [4]:
# 자동 정제 스크립트 (박스 우선 유지)

def clean_labels(label_dir):
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as f:
            lines = [l for l in f.readlines() if len(l.split()) == 5]
        
        with open(filepath, 'w') as f:
            f.writelines(lines)
        
clean_labels("C:/Users/user/Downloads/dataset/dataset/labels/train")
clean_labels("C:/Users/user/Downloads/dataset/dataset/labels/val")
clean_labels("C:/Users/user/Downloads/dataset/dataset/labels/test")

In [5]:
# 비어있는 파일이 있는 경우 성능 저하를 야기할 수 있으므로 비어 있는 label 파일과 이미지도 같이 삭제하는 파일

def clean_empty_labels(label_dir, image_dir=None, delete_images=False):
    label_dir = Path(label_dir)
    if image_dir:
        image_dir = Path(image_dir)

    removed_labels = 0
    removed_images = 0

    for label_file in label_dir.glob("*.txt"):
        if label_file.stat().st_size == 0:  # 파일이 비어있는지 확인
            # 라벨 파일 삭제
            label_file.unlink()
            removed_labels += 1

            if delete_images and image_dir:
                # 이미지 파일명 추정 (확장자 jpg, png 등)
                stem = label_file.stem
                for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                    img_file = image_dir / f"{stem}{ext}"
                    if img_file.exists():
                        img_file.unlink()
                        removed_images += 1
                        break

    print(f"삭제된 비어있는 라벨 파일: {removed_labels}개")
    if delete_images:
        print(f"함께 삭제된 이미지 파일: {removed_images}개")

        
# 라벨과 이미지 모두 삭제
clean_empty_labels(
    label_dir=r"C:\Users\user\Downloads\dataset\dataset\labels\train",
    image_dir=r"C:\Users\user\Downloads\dataset\dataset\images\train",
    delete_images=True
)

clean_empty_labels(
    label_dir=r"C:\Users\user\Downloads\dataset\dataset\labels\val",
    image_dir=r"C:\Users\user\Downloads\dataset\dataset\images\val",
    delete_images=True
)

clean_empty_labels(
    label_dir=r"C:\Users\user\Downloads\dataset\dataset\labels\test",
    image_dir=r"C:\Users\user\Downloads\dataset\dataset\images\test",
    delete_images=True
)

삭제된 비어있는 라벨 파일: 0개
함께 삭제된 이미지 파일: 0개
삭제된 비어있는 라벨 파일: 0개
함께 삭제된 이미지 파일: 0개
삭제된 비어있는 라벨 파일: 0개
함께 삭제된 이미지 파일: 0개


In [6]:
# 비어있는 파일이 존재하는지 확인하는 코드

def check_labels(label_path):
    empty_found = False  # 비어있는 파일이 발견되었는지 여부
    for lbl_file in Path(label_path).glob("*.txt"):
        with open(lbl_file) as f:
            lines = f.readlines()
            if not lines:
                print(f"Empty file: {lbl_file}")
                empty_found = True
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Invalid line in {lbl_file}: {line}")
    if not empty_found:
        print(f"[{label_path}] 비어있는 파일이 존재하지 않습니다.")

check_labels(r"C:\Users\user\Downloads\dataset\dataset\labels\train")
check_labels(r"C:\Users\user\Downloads\dataset\dataset\labels\val")
check_labels(r"C:\Users\user\Downloads\dataset\dataset\labels\test")

[C:\Users\user\Downloads\dataset\dataset\labels\train] 비어있는 파일이 존재하지 않습니다.
[C:\Users\user\Downloads\dataset\dataset\labels\val] 비어있는 파일이 존재하지 않습니다.
[C:\Users\user\Downloads\dataset\dataset\labels\test] 비어있는 파일이 존재하지 않습니다.


In [7]:
# YOLO 학습 과정에서 생성되는 캐시 파일 삭제하는 코드

cache_files = glob.glob(r"C:/Users/user/Downloads/dataset/dataset/labels/*.cache")
for f in cache_files:
    os.remove(f)
print(f"{len(cache_files)}개의 캐시 파일 삭제 완료")


2개의 캐시 파일 삭제 완료


In [8]:
# 클래스별 인스턴스 수를 확인할 수 있는 코드

def count_instance_per_class(name, label_dir):
    class_counts = defaultdict(int)

    for label_file in Path(label_dir).glob("*.txt"):
        with open(label_file) as f:
            for line in f:
                if line.strip():
                    class_id = int(line.split()[0])
                    class_counts[class_id] += 1

    print("{} 클래스별 인스턴스 수:".format(name))
    class_counts = dict(sorted(class_counts.items()))
    for cls, cnt in class_counts.items():
        print(f"Class {cls}: {cnt}개")

count_instance_per_class("train", r"C:\Users\user\Downloads\dataset\dataset\labels\train")
count_instance_per_class("val", r"C:\Users\user\Downloads\dataset\dataset\labels\val")
count_instance_per_class("test", r"C:\Users\user\Downloads\dataset\dataset\labels\test")

train 클래스별 인스턴스 수:
Class 0: 3665개
Class 1: 2596개
Class 2: 5132개
Class 3: 786개
val 클래스별 인스턴스 수:
Class 0: 408개
Class 1: 137개
Class 2: 251개
Class 3: 140개
test 클래스별 인스턴스 수:
Class 0: 207개
Class 1: 142개
Class 2: 249개
Class 3: 89개


In [ ]:
# 하이퍼파리미터 + 파라미터 튜닝 하는 부분

def objective(trial):
    try:
        lr0 = trial.suggest_float('lr0', 1e-4, 1e-1, log=True)
        batch = trial.suggest_categorical('batch', [8, 16, 32])
        mosaic = trial.suggest_float('mosaic', 0.0, 1.0)
        epochs = 50

        model = YOLO('yolov8n.pt')
        results = model.train(
            data=r"C:\Users\user\Downloads\dataset\dataset\data.yaml",
            epochs=epochs,
            lr0=lr0,
            batch=batch,
            mosaic=mosaic,
            imgsz=800,
            device='cuda',
            patience=20,
            verbose=False
        )

        try:
            map50 = results.metrics['metrics/val_map_0.5']
        except Exception:
            map50 = results.metrics.map_50

        return map50
    except Exception as e:
        print(f"Trial failed with error: {e}")
        return 0.0

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)
print("Best params:", study.best_params)


[I 2025-05-21 15:04:48,175] A new study created in memory with name: no-name-8969dcf6-5474-4262-a37a-5505cdb15ae9


Ultralytics 8.3.0  Python-3.10.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\user\Downloads\dataset\dataset\data.yaml, epochs=50, time=None, patience=20, batch=8, imgsz=800, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train36, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

C:\Users\user\anaconda3\lib\site-packages\ultralytics\utils\torch_utils.py:254: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:156.)
  fusedconv.weight.copy_(torch.mm(w_bn, w_conv).view(fusedconv.weight.shape))
C:\Users\user\anaconda3\lib\site-packages\ultralytics\utils\torch_utils.py:259: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this o

AMP: checks passed 


train: Scanning C:\Users\user\Downloads\dataset\dataset\labels\train... 6967 images, 0 backgrounds, 0 corrupt: 100%|███


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/871 [00:00<?, ?it/s]C:\Users\user\anaconda3\lib\site-packages\ultralytics\utils\loss.py:202: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:156.)
  pred_dist = pred_dist.view(b, a, 4, c // 4).softmax(3).matmul(self.proj.type(pred_dist.dtype))
      27/50      1.74G      1.067     0.9445      1.352         18        800: 100%|██████████| 871/871 [01:09<00:00, 1
                 Class     Images  Instances      Box(P          R      mA

                   all        752        936       0.86      0.708      0.814      0.553



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/871 [00:00<?, ?it/s]C:\Users\user\anaconda3\lib\site-packages\ultralytics\utils\loss.py:202: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:156.)
  pred_dist = pred_dist.view(b, a, 4, c // 4).softmax(3).matmul(self.proj.type(pred_dist.dtype))
      28/50      1.74G      1.069     0.9532      1.343         17        800:  44%|████▍     | 386/871 [00:30<00:38, 1

In [9]:
# 최적화 하이퍼파라미터와 파러미터 간 상관관계, 파라미터별 성능 분포 시각화 파트

# 1. 최적화 히스토리(각 trial별 best value 변화)
fig1 = plot_optimization_history(study)
fig1.show()

# 2. 하이퍼파라미터 중요도(어떤 파라미터가 성능에 영향이 큰지)
fig2 = plot_param_importances(study)
fig2.show()

# 3. 파라미터 간 상관관계(Parallel Coordinate)
fig3 = plot_parallel_coordinate(study)
fig3.show()

# 4. 파라미터별 성능 분포(Slice Plot)
fig4 = plot_slice(study)
fig4.show()

NameError: name 'study' is not defined

In [11]:
# 최적의 하이퍼파라미터로 테스트 진행

def test_model(model):
    # 1. 테스트 데이터셋 설정
    test_params = {
        'data': r"C:\Users\user\Downloads\dataset\dataset\data.yaml",
        'split': 'test',  
        'batch': 8,
        'name': 'disaster_detection_test',
        'plots': True,    
        'save_json': True
    }
    
    # 2. 테스트 실행
    results = model.val(**test_params)
    
    # 3. 결과 출력
    print("\n📊 최종 테스트 결과 리포트")
    print(f"mAP50: {results.box.map50:.4f}")
    print(f"Precision: {np.mean(results.box.p):.4f}")
    print(f"Recall: {np.mean(results.box.r):.4f}")
    
    return results

best_model_path = f"runs/detect/train35/weights/best.pt"
print("Best model path:", best_model_path)

model = YOLO(best_model_path)
test_result = test_model(model)

Best model path: runs/detect/train35/weights/best.pt
Ultralytics 8.3.0  Python-3.10.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 186 layers, 2,685,148 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning C:\Users\user\Downloads\dataset\dataset\labels\test... 561 images, 0 backgrounds, 0 corrupt: 100%|███████


val: New cache created: C:\Users\user\Downloads\dataset\dataset\labels\test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 71/71 [00:04


                   all        561        687      0.871      0.787      0.855      0.614
                 flood        135        207      0.872      0.727      0.782      0.634
                  fire        106        142      0.941      0.944      0.965      0.659
              accident        236        249      0.833      0.703      0.822      0.534
              sinkhole         84         89      0.838      0.775      0.851      0.631
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Saving runs\detect\disaster_detection_test7\predictions.json...
Results saved to runs\detect\disaster_detection_test7

📊 최종 테스트 결과 리포트
mAP50: 0.8550
Precision: 0.8711
Recall: 0.7872


In [ ]:
# 랜덤 5개 이미지에 대한 테스트 시각화

def visualize_extreme_diverse_predictions(model):
    test_images_dir = r"C:\Users\user\Downloads\dataset\dataset\images\test"
    all_images = os.listdir(test_images_dir)

    # 파일명에서 첫 단어로 그룹화
    image_groups = defaultdict(list)
    for img in all_images:
        if img:
            group_key = img.split('_')[0]  # 예: 'cat_01.jpg' → 'cat'
            image_groups[group_key].append(img)

    # 각 그룹에서 하나씩 뽑아 다양성 확보
    selected_images = [random.choice(group) for group in image_groups.values()]

    # 그룹이 5개 이상이면 랜덤하게 5개만 선택
    if len(selected_images) > 5:
        selected_images = random.sample(selected_images, 5)

    # 선택된 이미지에 대해 예측 및 결과 저장
    for img_name in selected_images:
        img_path = os.path.join(test_images_dir, img_name)
        results = model.predict(
            source=img_path,
            save=True,
            conf=0.5,  # 신뢰도 임계값
            save_dir='test_results'  # 예측 결과 저장 경로
        )
        print(f"{img_name} 예측 완료 → test_results 디렉토리 확인")

visualize_extreme_diverse_predictions(model)